In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

/Users/swithana/git/icicle_model_card/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!pip install tensorflow

     |████████████████████████████████| 208.8 MB 17.7 MB/s            
     |████████████████████████████████| 5.5 MB 29.9 MB/s            
     |████████████████████████████████| 394 kB 30.6 MB/s            
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached ml_dtypes-0.2.0-cp39-cp39-macosx_10_9_universal2.whl (1.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl (20.6 MB)
     |████████████████████████████████| 3.4 MB 32.8 MB/s            
  Using cached h5py-3.10.0-cp39-cp39-macosx_11_0_arm64.whl (2.7 MB)
     |████████████████████████████████| 441 kB 33.2 MB/s            
  Using cached wrapt-1.14.1-cp39-cp39-macosx_11_0_arm64.whl (35 kB)
     |████████████████████████████████| 9.7 MB 40.5 MB/s            
     |████████████████████████████████| 1.7 MB 28.8 MB/s           

In [2]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "martial-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"]
data = pd.read_csv('/Users/swithana/git/icicle_model_card/examples/notebooks/data/adult/adult.data', names=columns)

In [3]:
data.head()

,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pre-processing

In [4]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

# Convert categorical variables to one-hot encoding
data = pd.get_dummies(data, drop_first=True, dtype=float)

data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = data.drop('income', axis=1).values
y = data['income'].values

print(data.columns.tolist())

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'martial-status_ Married-AF-spouse', 'martial-status_ Married-civ-spouse', 'martial-status_ Married-spouse-absent', 'martial-status_ Never-married', 'martial-status_ Separated', 'martial-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupa

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape

(26048, 100)

### Model Training

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
733/733 [==============================] - 1s 824us/step - loss: 122.1293 - accuracy: 0.6697 - val_loss: 91.1584 - val_accuracy: 0.8019
Epoch 2/100
733/733 [==============================] - 0s 667us/step - loss: 56.1731 - accuracy: 0.6815 - val_loss: 9.6071 - val_accuracy: 0.7992
Epoch 3/100
733/733 [==============================] - 0s 646us/step - loss: 27.8379 - accuracy: 0.6818 - val_loss: 4.8409 - val_accuracy: 0.7969
Epoch 4/100
733/733 [==============================] - 0s 622us/step - loss: 12.1377 - accuracy: 0.6855 - val_loss: 9.8526 - val_accuracy: 0.7900
Epoch 5/100
733/733 [==============================] - 0s 635us/step - loss: 7.6852 - accuracy: 0.6828 - val_loss: 10.0492 - val_accuracy: 0.2365
Epoch 6/100
733/733 [==============================] - 0s 631us/step - loss: 3.1837 - accuracy: 0.6853 - val_loss: 1.0472 - val_accuracy: 0.2626
Epoch 7/100
733/733 [==============================] - 0s 631us/step - loss: 2.3292 - accuracy: 0.6938 - val_loss: 2.0444 -

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)

204/204 [==============================] - 0s 446us/step - loss: 0.4956 - accuracy: 0.7981


In [21]:
y_pred = model.predict(X_test)

204/204 [==============================] - 0s 404us/step


## Model Card Generation

In [13]:
from icicle_model_card.icicle_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis, validate_mc, Metric, save_mc
import json

In [16]:
mc = ModelCard(
            name="UCI Adult Data Analysis model using Tensorflow",
            version="0.1",
            short_description="UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
            full_description="We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
            keywords="uci adult, tensorflow, explainability, fairness, patra",
            author="Sachith Withana",
            input_type="Tabular",
            category="Classification"
        )

mc.input_data = 'https://archive.ics.uci.edu/dataset/2/adult'
mc.output_data = 'https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01'

In [40]:
def remove_nulls(model_structure):
    if isinstance(model_structure, dict):
        return {k: remove_nulls(v) for k, v in model_structure.items() if v is not None and remove_nulls(v)}
    elif isinstance(model_structure, list):
        return [remove_nulls(v) for v in model_structure if v is not None and remove_nulls(v) != []]
    else:
        return model_structure

def clean_model_structure(model_structure):
    json_structure = json.loads(model_structure)
    cleaned_structure = remove_nulls(json_structure)
    return cleaned_structure

In [41]:
model_metrics = []
model_metrics.append(Metric("Test loss", loss))
model_metrics.append(Metric("Epochs", 100))
model_metrics.append(Metric("Batch Size", 32))
model_metrics.append(Metric("Optimizer", "Adam"))
model_metrics.append(Metric("Learning Rate", 0.0001))
model_metrics.append(Metric("Input Shape", "(26048, 100)"))


ai_model = AIModel(
            name="Income prediction tensorflow model",
            version="0.1",
            description="Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
            owner="Sachith Withana",
            location="https://github.iu.edu/d2i/uci_adult/tensorflow_model",
            license="BSD-3 Clause",
            framework="tensorflow",
            foundational_model="None",
            model_type="dnn",
            test_accuracy=accuracy,
            model_structure = clean_model_structure(model.to_json())
        )
ai_model.metrics = model_metrics
mc.ai_model = ai_model

In [42]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "Classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
       

In [43]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()
mc.populate_bias(X_test, y_test, y_pred, "gender", X_test[:, 58], model)

204/204 [==============================] - 0s 368us/step


DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [44]:
x_columns = data.columns.tolist()
x_columns.remove('income')

mc.populate_xai(X_test, x_columns, model, 10)

PermutationExplainer explainer: 6514it [04:01, 26.05it/s]                                                                                                                                                                                                                                                                                                                                               


In [45]:
print(mc.bias_analysis)

{'demographic_parity_diff': 0.04268373096686717, 'equal_odds_difference': 0.04169312983955298}


In [46]:
print(mc.xai_analysis)

{'capital-gain': 0.04943593070968624, 'fnlwgt': 0.007977424863480671, 'capital-loss': 0.0016644893624441271, 'age': 2.839242701838145e-05, 'martial-status_ Married-civ-spouse': 1.6070720708102193e-05, 'relationship_ Not-in-family': 1.3571421730930955e-05, 'martial-status_ Never-married': 1.1598601625075779e-05, 'education-num': 1.0616391369266047e-05, 'hours-per-week': 9.01362025913249e-06, 'occupation_ Exec-managerial': 8.548350777930122e-06}


In [47]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "Classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
       

In [48]:
save_mc(mc, "/Users/swithana/git/icicle_model_card/examples/model_cards/tesorflow_adult_nn_MC.json")